In [1]:
import os
os.makedirs('new_ndh', exist_ok=True)
files = [x for x in os.listdir('ndh') if 'test' not in x]

In [3]:
import json
from collections import defaultdict
object_locations = defaultdict(list)
for file in files:
    path = os.path.join('ndh', file)
    with open(path, 'r') as f:
        data = json.load(f)
        for ins in data:
            scan = ins['scan']
            end_panos = ins['end_panos']
            target = ins['target']
            scan_end_panos = [f"{scan}_{x}" for x in end_panos]
            if scan_end_panos not in object_locations[target]:
                object_locations[target].append(scan_end_panos)

ndh_objects = list(object_locations.keys())
with open("coco.txt", 'r') as f:
    coco_objects = f.read().splitlines()
coco_objects = [x.lower() for x in coco_objects]

with open("lvis_v1_val.json", 'r') as f:
    lvis = json.load(f)
lvis_categories = lvis['categories']
lvis_objects = []
for x in lvis_categories:
    lvis_objects += [y.lower() for y in x['synonyms']]

# calculate intersection
ndh_objects = set(ndh_objects)
coco_objects = set(coco_objects)
lvis_objects = set(lvis_objects)
lvis_intersection = ndh_objects.intersection(lvis_objects)
coco_intersection = ndh_objects.intersection(coco_objects).difference(lvis_intersection)
ndh_left = ndh_objects - lvis_intersection



In [4]:
lvis_locations = {}
for obj in lvis_intersection:
    lvis_locations[obj] = object_locations[obj]
with open("lvis_location.json", 'w') as f:
    json.dump(lvis_locations, f, indent=4)
print(len(lvis_locations))

46


In [5]:
need_label = {}
for obj in ndh_left:
    need_label[obj] = object_locations[obj]
with open("need_label.json", 'w') as f:
    json.dump(need_label, f, indent=4)
print(len(need_label))

34
